 # Dr. Kativa Agarwal RNA-seq analysis of VK2 cells


# RNASeq Data Integration

* Daisy Chilin-Fuentes, CCBB (dchilinfuentes@ucsd.edu)
* Based on upstream analysis by Daisy Chilin-Fuentes, CCBB (dchilinfuentes@ucsd.edu)


* Modeled on "RNA-seq analysis is easy as 1-2-3 with limma, Glimma and edgeR" ([1](#Citations))

## Table of Contents
* [Background](#Background)
* [Introduction](#Introduction)
* [Parameter Input](#Parameter-Input)
* [Library Import](#Library-Import)
* [Data Import](#Data-Import)
    * [Count Data](#Count-Data)
    * [Metadata](#Metadata)
    * [Annotations](#Annotations)
* [Gene Separation By Coding Status](#Gene-Separation-By-Coding-Status)
* [Data Integration](#Data-Integration)
* [Annotation Integration](#Annotation-Integration)
* [Summary](#Summary)
* [Citations](#Citations)
* [Appendix: R Session Info](#Appendix:-R-Session-Info)


## Background

The count data analyzed in this notebook were produced by the upstream analysis of Daisy Chilin-Fuentes of CCBB, who received raw sequencing data and performed quality control, trimming, alignment, and quantification of reads.


[Table of Contents](#Table-of-Contents)

## Introduction

This notebook takes in per-gene-per-sample count data (prepared either externally or by the  "RNASeq_RSEM_QC_and_Counts_Preparation" notebook) and per-sample metadata RNASeq data, and uses the edgeR ([2](#Citations)) Bioconductor ([3](#Citations)) package written in R ([4](#Citations)) to integrate and annotate these inputs in preparation for data exploration and preprocessing.

[Table of Contents](#Table-of-Contents)

## Parameter Input

In [1]:
gProjectName = "Agarwal-Lewis_bulkRNAseq"
gGeneCountsFilename = "counts.txt"
gMetadataFilename = "KavitaSamplesMetadata.tsv"


gAnnotationsRdataFilename = "Homo_sapiens_GRCh38p13_gencodev38_ANNOT.Rdata"

In [3]:
gSourceDir = "./src/" # note trailing slash here but not below
gOutputDir = "../secondary_analysis_results"
gReferenceDir = "../reference"
gInterimDir = "../interim"
gGeneCountsFp = file.path(gOutputDir, "RSEM_QC_and_Counts_Preparation", gGeneCountsFilename)
gMetadataFp = file.path(gReferenceDir, gMetadataFilename)

gOutputDir_nb1 <- paste0(gOutputDir, "/data_integration")
dir.create(paste0(gOutputDir, "/data_integration")) 


Warning message in dir.create(paste0(gOutputDir, "/data_integration")):
“'../secondary_analysis_results/data_integration' already exists”


In [4]:
# Import shared source code to load and save previous notebooks' environments:
source(paste0(gSourceDir, "ChainedNotebookSupport.R"))

Populate the run name parameter automatically to ensure that outputs from different runs do not overwrite each other:

In [5]:
gRunName = format(Sys.time(), "%Y%m%d%H%M%S")
gRunName

[1] "20230302161642"

In [48]:
# Create subdirectory with timestamp to keep multiple iterations separated
gOutputDir_nb1 <- paste0(gOutputDir_nb1, "/", gRunName)
dir.create(gOutputDir_nb1) 
gOutputDir_nb1

[1] "../secondary_analysis_results/data_integration/20230302161642/20230302161642"

[Table of Contents](#Table-of-Contents)

## Library Import

Import the necessary R, Bioconductor, and CCBB libraries for the analysis:

In [7]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

In [8]:
#BiocManager::install("edgeR", version = "3.8")

In [9]:
#BiocManager::install("Homo.sapiens", version = "3.8")

In [49]:
library(Homo.sapiens)
gOrganismPackage = Homo.sapiens

In [50]:
library(edgeR)

[Table of Contents](#Table-of-Contents)


## Data Import

### Count Data

Import the count data file in which rows are genes identifiers, columns are sample identifiers, and row/column intersections contain the number of counts for the relevant gene in the relevant sample:

In [51]:
# Read in counts file containing info on all samples and genes
gUnorderedGeneCountsDf <- read.csv(gGeneCountsFp, sep="\t", stringsAsFactors=FALSE, row.names=1)
dim(gUnorderedGeneCountsDf)

[1] 60605    12

In [52]:
head(gUnorderedGeneCountsDf)

,VK2_NanH2_PolyB25_1h_5,VK2_NanH2_PolyB25_1h_6,VK2_NanH2_PolyB25_1h_7,VK2_NanH2_PolyB25_1h_8,VK2_NanH2_PolyB25_2h_1,VK2_NanH2_PolyB25_2h_2,VK2_NanH2_PolyB25_2h_3,VK2_NanH2_PolyB25_2h_4,VK2_Pet28a_PolyB25_2h_10,VK2_Pet28a_PolyB25_2h_11,VK2_Pet28a_PolyB25_2h_12,VK2_Pet28a_PolyB25_2h_9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003.15,502.00,416.00,424.00,497.00,356.00,542.00,364.00,299.00,387.00,465.00,366.00,323.00
ENSG00000000005.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419.14,2340.00,2232.04,2253.00,2573.36,2080.11,3080.00,2206.80,1721.00,2084.00,2412.42,2170.07,1904.05
ENSG00000000457.14,153.65,94.84,112.33,130.23,113.88,119.35,85.84,102.65,98.34,120.36,91.56,119.77
ENSG00000000460.17,254.35,200.16,242.67,292.77,181.12,285.65,181.16,209.35,189.66,190.64,204.44,234.23
ENSG00000000938.13,10.00,6.00,6.00,9.00,4.00,21.00,10.00,18.00,13.00,17.00,13.00,10.00


In [53]:
detectParRecords = function(geneCountsDf){
    gene_names <- rownames(geneCountsDf)
    PAR_genes <- gene_names[grep("_PAR_", gene_names)] 
    if (length(PAR_genes) == 0){
        print("No PAR genes detected; analysis can proceed.")
    } else {
        print("ERROR: PAR genes found.  These must be removed before continuing analysis.")
    }
    return(PAR_genes)
}

In [54]:
detectParRecords(gUnorderedGeneCountsDf)

[1] "No PAR genes detected; analysis can proceed."


character(0)

No assumption is made that the columns (samples) of the gene count file are currently ordered in the order desirable for the differential expression analysis.

[Table of Contents](#Table-of-Contents)

### Metadata

> For downstream analysis, sample-level information related to the experimental design needs to be associated with the columns of the counts matrix. This should include experimental variables, both biological and technical, that could have an effect on expression levels. Examples [could] include cell type (basal, LP and ML in this experiment), genotype (wild-type, knock-out), phenotype (disease status, sex, age), sample treatment (drug, control) and batch information (date experiment was performed if samples were collected and analysed at distinct time points) to name just a few. ([1](#Citations))

Import a metadata file in which rows are sample identifiers, columns are metadata features (e.g., subject id, time point, etc) and row/column intersections contain the value of the relevant feature for the relevant sample:

In [55]:
#Read in metadata
gMetadataDf <- read.csv(gMetadataFp, stringsAsFactors=FALSE, sep = "\t")
dim(gMetadataDf)

[1] 12  6

In [56]:
head(gMetadataDf)

,Sample.Name,Sample.Code,Reference.Genome..Organism.,Gender,Condition,Time.point
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,VK2_NanH2_PolyB25_2h_1,K001,Human,Female,NanH2 treated,2 hour
2,VK2_NanH2_PolyB25_2h_2,K002,Human,Female,NanH2 treated,2 hour
3,VK2_NanH2_PolyB25_2h_3,K003,Human,Female,NanH2 treated,2 hour
4,VK2_NanH2_PolyB25_2h_4,K004,Human,Female,NanH2 treated,2 hour
5,VK2_NanH2_PolyB25_1h_5,K005,Human,Female,NanH2 treated,1 hour
6,VK2_NanH2_PolyB25_1h_6,K006,Human,Female,NanH2 treated,1 hour


In [59]:
#remove spaces
gMetadataDf$Condition <- gsub(" ", "_", gMetadataDf$Condition)
gMetadataDf$Time.point <- gsub("2 hour", "2H", gMetadataDf$Time.point)
gMetadataDf$Time.point <- gsub("1 hour", "1H", gMetadataDf$Time.point)


#add another col
gMetadataDf$Condition_Time <- paste(gMetadataDf$Condition, gMetadataDf$Time.point, sep="_")
head(gMetadataDf)

,Sample.Name,Sample.Code,Reference.Genome..Organism.,Gender,Condition,Time.point,Condition_Time
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,VK2_NanH2_PolyB25_2h_1,K001,Human,Female,NanH2_treated,2H,NanH2_treated_2H
2,VK2_NanH2_PolyB25_2h_2,K002,Human,Female,NanH2_treated,2H,NanH2_treated_2H
3,VK2_NanH2_PolyB25_2h_3,K003,Human,Female,NanH2_treated,2H,NanH2_treated_2H
4,VK2_NanH2_PolyB25_2h_4,K004,Human,Female,NanH2_treated,2H,NanH2_treated_2H
5,VK2_NanH2_PolyB25_1h_5,K005,Human,Female,NanH2_treated,1H,NanH2_treated_1H
6,VK2_NanH2_PolyB25_1h_6,K006,Human,Female,NanH2_treated,1H,NanH2_treated_1H


In [60]:
gSampleNames = gMetadataDf[["Sample.Name"]]

Check the dimensions of the count data and the metadata to ensure that the count dataframe has the same number of columns (samples) as the metadata dataframe has rows (again, samples), and that the sample names are the same in both: 

In [61]:
dim(gUnorderedGeneCountsDf)
dim(gMetadataDf)

all(colnames(gUnorderedGeneCountsDf) %in% gSampleNames)

[1] 60605    12

[1] 12  7

[1] TRUE

Assume that the order of the samples shown in the metadata is the desired order, and reorder the columns in the counts table to match it:

In [62]:
gGeneCountsDf = gUnorderedGeneCountsDf[gSampleNames]
head(gGeneCountsDf)

,VK2_NanH2_PolyB25_2h_1,VK2_NanH2_PolyB25_2h_2,VK2_NanH2_PolyB25_2h_3,VK2_NanH2_PolyB25_2h_4,VK2_NanH2_PolyB25_1h_5,VK2_NanH2_PolyB25_1h_6,VK2_NanH2_PolyB25_1h_7,VK2_NanH2_PolyB25_1h_8,VK2_Pet28a_PolyB25_2h_9,VK2_Pet28a_PolyB25_2h_10,VK2_Pet28a_PolyB25_2h_11,VK2_Pet28a_PolyB25_2h_12
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003.15,356.00,542.00,364.00,299.00,502.00,416.00,424.00,497.00,323.00,387.00,465.00,366.00
ENSG00000000005.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419.14,2080.11,3080.00,2206.80,1721.00,2340.00,2232.04,2253.00,2573.36,1904.05,2084.00,2412.42,2170.07
ENSG00000000457.14,113.88,119.35,85.84,102.65,153.65,94.84,112.33,130.23,119.77,98.34,120.36,91.56
ENSG00000000460.17,181.12,285.65,181.16,209.35,254.35,200.16,242.67,292.77,234.23,189.66,190.64,204.44
ENSG00000000938.13,4.00,21.00,10.00,18.00,10.00,6.00,6.00,9.00,10.00,13.00,17.00,13.00


If the count file gene identifiers do NOT include version numbers (e.g., the ".4" part in a gene identifier like "ENSG00000268020.4"), then it is necessary to truncate the version information from the public annotation data to be used below in order to match the annotation data gene identifiers to the count file gene identifiers.  Set the flag for version removal accordingly:

In [63]:
gRemoveVersion <- FALSE


[Table of Contents](#Table-of-Contents)

###  Annotations

If a previously created file of the gene annotations has been provided, load it:

In [64]:

if (!is.null(gAnnotationsRdataFilename)) {
    gAnnotationsRdataFp = file.path(gReferenceDir, gAnnotationsRdataFilename)  
    
    # Import the R data object containing gene annotations and load its dataframe into a variable:
    gAnnotationEnv = loadToEnvironment(gAnnotationsRdataFp)
    gGeneTypeAnnotationsDf = gAnnotationEnv$ANNOT
    
    head(gGeneTypeAnnotationsDf)
} else {
    print("No annotations provided.")
}

,gene_type,gene_id,transcript_id
,<chr>,<chr>,<chr>
1,transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000456328.2
2,transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000450305.2
3,unprocessed_pseudogene,ENSG00000227232.5,ENST00000488147.1
4,miRNA,ENSG00000278267.1,ENST00000619216.1
5,lncRNA,ENSG00000243485.5,ENST00000473358.1
6,lncRNA,ENSG00000243485.5,ENST00000469289.1


[Table of Contents](#Table-of-Contents)

## Gene Separation By Coding Status

Gene annotations are records of each gene's identifier and symbol, where the gene begins and ends on the genome sequence, and whether it is anticipated to be a coding gene or not.  There are multiple sources of gene annotations.

Here we use the human gene annotations from the Gencode project, Release 43 (GRCh38.p13). 



In [65]:
splitGeneCountsByCodingStatus = function(geneCountDf, gtfDf, removeVersion=FALSE){
    #Subset GTF by protein coding and noncoding
    ANNOT_protein_coding <- subset(gtfDf, gene_type == "protein_coding")
#     ANNOT_ncRNA <- subset(gtfDf, gene_type %in% c("lincRNA", "antisense", "processed_transcript","sense_overlapping", "sense_intronic"))
    ANNOT_ncRNA <- subset(gtfDf, gene_type %in% c("lncRNA", "antisense", "processed_transcript","sense_overlapping", "sense_intronic"))

    #make list of IDs to query
    protein_coding_ids <- ANNOT_protein_coding$gene_id
    ncRNA_ids <- ANNOT_ncRNA$gene_id
    
    if (removeVersion){
        protein_coding_ids <- removeAccessionVersion(protein_coding_ids)
        ncRNA_ids <- removeAccessionVersion(ncRNA_ids)        
    }

    #subset geneCounts
    geneCount_protein_coding <- subset(geneCountDf, row.names(geneCountDf) %in% protein_coding_ids)
    geneCount_ncRNA <- subset(geneCountDf, row.names(geneCountDf) %in% ncRNA_ids)
    return(list(codingGeneCountDf=geneCount_protein_coding, noncodingGeneCountDf=geneCount_ncRNA))
}

removeAccessionVersion = function(accessionVector){
    return (gsub("\\..*","",accessionVector))
}

writeSubsetCounts = function(subsetCountsDf, outputDir, runName, fileSuffix){
    fileName = sprintf(fileSuffix, runName)
    write.csv(subsetCountsDf, file.path(outputDir, fileName))
    print(paste0("Output file: ",fileName))
}

writeSubsetsCounts = function(splitGeneCountDfsList, outputDir, runName){
    writeSubsetCounts(splitGeneCountDfsList$codingGeneCountDf, outputDir, runName,"%s_raw_pc_genes_counts.csv")
    writeSubsetCounts(splitGeneCountDfsList$noncodingGeneCountDf, outputDir, runName,"%s_raw_nc_genes_counts.csv")
}

Split the count data into coding and non-coding subsets, and extract each subset into a file based on the annotation file provided in the input parameters:

In [66]:
gSplitGeneCountDfsList = splitGeneCountsByCodingStatus(gGeneCountsDf, gGeneTypeAnnotationsDf, gRemoveVersion)

In [67]:
dim(gGeneCountsDf)
dim(gSplitGeneCountDfsList$codingGeneCountDf)
dim(gSplitGeneCountDfsList$noncodingGeneCountDf)

[1] 60605    12

[1] 19937    12

[1] 16876    12

 Of the original 60,605 Ensembl genes in the dataset, 19,937 are known coding genes.  

In [68]:
writeSubsetsCounts(gSplitGeneCountDfsList, gOutputDir_nb1, gRunName)

[1] "Output file: 20230302161642_raw_pc_genes_counts.csv"
[1] "Output file: 20230302161642_raw_nc_genes_counts.csv"


[Table of Contents](#Table-of-Contents)

## Data Integration



Integrate the count data and the metadata into an edgeR DGEList object for use in downstream analysis:

> Our DGEList-object contains a samples data frame that stores both ... group ... and batch ... information, each of which consists of ... distinct levels. Note that within x$samples, library sizes are automatically calculated for each sample and normalisation factors are set to 1. ([1](#Citations))

In [69]:
gGeneType = "all"
gRelevantGeneCountsDf <- gGeneCountsDf

In [70]:
# create a DGEList object
makeDgeList = function(countsDf, metadataDf, groupColName){
    # remove the accession version (.##etc) from the ensembl gene id
    id_list <- gsub("[.].*$","", row.names(countsDf))
    row.names(countsDf) <- id_list

    x <- DGEList(counts = countsDf, lib.size = colSums(countsDf),
    norm.factors = rep(1,ncol(countsDf)), samples = metadataDf,
        group = metadataDf[[groupColName]], genes = NULL, remove.zeros = FALSE)
    return(x)
}

In [71]:
gGroupCategory = "Condition_Time" # e.g., "day"

In [72]:
gDgeList = makeDgeList(gRelevantGeneCountsDf, gMetadataDf, gGroupCategory)
names(gDgeList)

[1] "counts"  "samples"

As a sanity-check, look at representative content from the DGEList:

In [73]:
head(gDgeList$counts)
head(gDgeList$samples)

,VK2_NanH2_PolyB25_2h_1,VK2_NanH2_PolyB25_2h_2,VK2_NanH2_PolyB25_2h_3,VK2_NanH2_PolyB25_2h_4,VK2_NanH2_PolyB25_1h_5,VK2_NanH2_PolyB25_1h_6,VK2_NanH2_PolyB25_1h_7,VK2_NanH2_PolyB25_1h_8,VK2_Pet28a_PolyB25_2h_9,VK2_Pet28a_PolyB25_2h_10,VK2_Pet28a_PolyB25_2h_11,VK2_Pet28a_PolyB25_2h_12
ENSG00000000003,356.00,542.00,364.00,299.00,502.00,416.00,424.00,497.00,323.00,387.00,465.00,366.00
ENSG00000000005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419,2080.11,3080.00,2206.80,1721.00,2340.00,2232.04,2253.00,2573.36,1904.05,2084.00,2412.42,2170.07
ENSG00000000457,113.88,119.35,85.84,102.65,153.65,94.84,112.33,130.23,119.77,98.34,120.36,91.56
ENSG00000000460,181.12,285.65,181.16,209.35,254.35,200.16,242.67,292.77,234.23,189.66,190.64,204.44
ENSG00000000938,4.00,21.00,10.00,18.00,10.00,6.00,6.00,9.00,10.00,13.00,17.00,13.00


,group,lib.size,norm.factors,Sample.Name,Sample.Code,Reference.Genome..Organism.,Gender,Condition,Time.point,Condition_Time
,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
VK2_NanH2_PolyB25_2h_1,NanH2_treated_2H,22490889,1,VK2_NanH2_PolyB25_2h_1,K001,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_2h_2,NanH2_treated_2H,31232480,1,VK2_NanH2_PolyB25_2h_2,K002,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_2h_3,NanH2_treated_2H,21901342,1,VK2_NanH2_PolyB25_2h_3,K003,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_2h_4,NanH2_treated_2H,19637114,1,VK2_NanH2_PolyB25_2h_4,K004,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_1h_5,NanH2_treated_1H,27468016,1,VK2_NanH2_PolyB25_1h_5,K005,Human,Female,NanH2_treated,1H,NanH2_treated_1H
VK2_NanH2_PolyB25_1h_6,NanH2_treated_1H,24935803,1,VK2_NanH2_PolyB25_1h_6,K006,Human,Female,NanH2_treated,1H,NanH2_treated_1H


[Table of Contents](#Table-of-Contents)

## Annotation Integration

Next, extend the DGEList object with annotation information about the genes that have count data with symbol and EntrezId information, based upon their Ensembl ids.

> A second data frame named genes in the DGEList-object is used to store gene-level information associated with rows of the counts matrix. This information can be retrieved using organism specific packages such as Mus.musculus (Bioconductor Core Team 2016b) for mouse (or Homo.sapiens (Bioconductor Core Team 2016a) for human) ....
>
> The type of information that can be retrieved includes gene symbols, gene names, chromosome names and locations, Entrez gene IDs, Refseq gene IDs and Ensembl gene IDs to name just a few. .... Mus.musculus [and other organism-specific packages] packages information from various sources and allows users to choose between many different gene IDs as the key. ([1](#Citations))

In [74]:
getGeneDf = function(dgeList, organismPackage){
    geneid <-  rownames(dgeList)
    genes <- select(organismPackage, keys=geneid, columns=c("SYMBOL", "ENSEMBL", "ENTREZID"), 
                    keytype="ENSEMBL")
    return(genes)
}

In [75]:
gRawGenesDf = getGeneDf(gDgeList, gOrganismPackage)
dim(gRawGenesDf)

'select()' returned 1:many mapping between keys and columns



[1] 60846     3

In [76]:
# Add gene type to gRawGenesDf
gGeneTypeAnnotationsDf.rmdec <- gGeneTypeAnnotationsDf
gGeneTypeAnnotationsDf.rmdec$gene_id <- gsub("\\..*","",gGeneTypeAnnotationsDf.rmdec$gene_id)
gRawGenesDf$gene_type <- gGeneTypeAnnotationsDf$gene_type[match(gRawGenesDf$ENSEMBL, gGeneTypeAnnotationsDf.rmdec$gene_id)]

In [77]:
head(gRawGenesDf)

,ENSEMBL,ENTREZID,SYMBOL,gene_type
,<chr>,<chr>,<chr>,<chr>
1,ENSG00000000003,7105,TSPAN6,protein_coding
2,ENSG00000000005,64102,TNMD,protein_coding
3,ENSG00000000419,8813,DPM1,protein_coding
4,ENSG00000000457,57147,SCYL3,protein_coding
5,ENSG00000000460,55732,C1orf112,protein_coding
6,ENSG00000000938,2268,FGR,protein_coding


> [G]ene IDs may not map one-to-one to the gene information of interest. It is important to check for duplicated gene IDs. ([1](#Citations))

Examine how many records in the annotation dataset have the same id (for the gene identifier type--either ENSEMBL or ENTREZ--set below) as another record occurring earlier in the dataset:

In [78]:
gGeneIdCol <- "ENSEMBL"
# gGeneIdCol <- "ENTREZ"

In [79]:
gDuplicatesMask = duplicated(gRawGenesDf[[gGeneIdCol]])
sum(gDuplicatesMask) # Sum counts only those with a value of TRUE

[1] 241

Note that this sum includes only the second (or greater) instances of records for each gene id; the first record for each gene id is not included in this duplicate set.

Write a file of the duplicate records that can be examined if desired: 

In [80]:
writeOutRemovedDuplicates = function(countsDf, duplicatesMask, outputDir, runName, geneType){
    fileName = sprintf("%s_duplicated_%s_genes_records.csv",runName, geneType)
    duplicatedCountsDf = countsDf[duplicatesMask,]
    write.csv(duplicatedCountsDf, file.path(outputDir, fileName))
    print(paste0("Output file: ",fileName))
}

In [81]:
writeOutRemovedDuplicates(gRawGenesDf, gDuplicatesMask, gOutputDir_nb1, gRunName, gGeneType)

[1] "Output file: 20230302161642_duplicated_all_genes_records.csv"


 As a basic approach, duplicate records for gene ids already existing in the annotation are removed

In [82]:
gDeduplicatedGenesDf = gRawGenesDf[!duplicated(gRawGenesDf[[gGeneIdCol]]),]

After deduplication, check the dimensions of the count data and the gene annotation data to ensure that the count dataframe has the same number of rows (genes) as the gene annotation dataframe has rows (again, genes), and that the gene names are the same in both:

In [83]:
dim(gDgeList$counts)
dim(gDeduplicatedGenesDf)

all(rownames(gDgeList$counts) %in% gDeduplicatedGenesDf[[gGeneIdCol]])

[1] 60605    12

[1] 60605     4

[1] TRUE

Add the annotation information to the DGEList object:

In [84]:
gDgeList$genes = gDeduplicatedGenesDf
names(gDgeList)

[1] "counts"  "samples" "genes"

As a sanity-check, look at representative content from the DGEList:

In [85]:
head(gDgeList$counts)
head(gDgeList$samples)
head(gDgeList$genes)

,VK2_NanH2_PolyB25_2h_1,VK2_NanH2_PolyB25_2h_2,VK2_NanH2_PolyB25_2h_3,VK2_NanH2_PolyB25_2h_4,VK2_NanH2_PolyB25_1h_5,VK2_NanH2_PolyB25_1h_6,VK2_NanH2_PolyB25_1h_7,VK2_NanH2_PolyB25_1h_8,VK2_Pet28a_PolyB25_2h_9,VK2_Pet28a_PolyB25_2h_10,VK2_Pet28a_PolyB25_2h_11,VK2_Pet28a_PolyB25_2h_12
ENSG00000000003,356.00,542.00,364.00,299.00,502.00,416.00,424.00,497.00,323.00,387.00,465.00,366.00
ENSG00000000005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419,2080.11,3080.00,2206.80,1721.00,2340.00,2232.04,2253.00,2573.36,1904.05,2084.00,2412.42,2170.07
ENSG00000000457,113.88,119.35,85.84,102.65,153.65,94.84,112.33,130.23,119.77,98.34,120.36,91.56
ENSG00000000460,181.12,285.65,181.16,209.35,254.35,200.16,242.67,292.77,234.23,189.66,190.64,204.44
ENSG00000000938,4.00,21.00,10.00,18.00,10.00,6.00,6.00,9.00,10.00,13.00,17.00,13.00


,group,lib.size,norm.factors,Sample.Name,Sample.Code,Reference.Genome..Organism.,Gender,Condition,Time.point,Condition_Time
,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
VK2_NanH2_PolyB25_2h_1,NanH2_treated_2H,22490889,1,VK2_NanH2_PolyB25_2h_1,K001,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_2h_2,NanH2_treated_2H,31232480,1,VK2_NanH2_PolyB25_2h_2,K002,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_2h_3,NanH2_treated_2H,21901342,1,VK2_NanH2_PolyB25_2h_3,K003,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_2h_4,NanH2_treated_2H,19637114,1,VK2_NanH2_PolyB25_2h_4,K004,Human,Female,NanH2_treated,2H,NanH2_treated_2H
VK2_NanH2_PolyB25_1h_5,NanH2_treated_1H,27468016,1,VK2_NanH2_PolyB25_1h_5,K005,Human,Female,NanH2_treated,1H,NanH2_treated_1H
VK2_NanH2_PolyB25_1h_6,NanH2_treated_1H,24935803,1,VK2_NanH2_PolyB25_1h_6,K006,Human,Female,NanH2_treated,1H,NanH2_treated_1H


,ENSEMBL,ENTREZID,SYMBOL,gene_type
,<chr>,<chr>,<chr>,<chr>
1,ENSG00000000003,7105,TSPAN6,protein_coding
2,ENSG00000000005,64102,TNMD,protein_coding
3,ENSG00000000419,8813,DPM1,protein_coding
4,ENSG00000000457,57147,SCYL3,protein_coding
5,ENSG00000000460,55732,C1orf112,protein_coding
6,ENSG00000000938,2268,FGR,protein_coding


[Table of Contents](#Table-of-Contents)

## Summary

**Gene annotations**
* Human gene annotations were taken from the Gencode project, Release 43 (GRCh38.p13).

**Gene type filtering**
* This analysis was includes protein-coding genes and non-coding genes.  Of the original 60,605 Ensembl genes in the dataset, 19,937 are known coding genes.  


Save the workspace objects for future reference:

In [86]:
writeWorkspaceImage(gInterimDir, paste0(gRunName,"_data_integration"))

[1] "Output file: 20230302161642_data_integration.RData"


[Table of Contents](#Table-of-Contents)

## Citations

1. Law CW, Alhamdoosh M, Su S, Smyth GK, Ritchie ME. RNA-seq analysis is easy as 1-2-3 with limma, Glimma and edgeR. Version 2. F1000Res. 2016 Jun 17 [revised 2016 Jan 1];5:1408.
2. Robinson MD, McCarthy DJ and Smyth GK (2010). edgeR: a Bioconductor package for differential expression analysis of digital gene expression data. Bioinformatics 26, 139-140.
3. Huber W, Carey VJ, Gentleman R, Anders S, Carlson M, Carvalho BS, Bravo HC, Davis S, Gatto L, Girke T, Gottardo R, Hahne F, Hansen KD, Irizarry RA, Lawrence M, Love MI, MacDonald J, Obenchain V, Oleś AK, Pagès H, Reyes A, Shannon P, Smyth GK, Tenenbaum D, Waldron L, Morgan M. Orchestrating high-throughput genomic analysis with Bioconductor. Nat Methods. 2015 Feb;12(2):115-21.
4. R Core Team (2016). R: A language and environment for statistical computing. R Foundation for Statistical Computing, Vienna, Austria. URL https://www.R-project.org/.

[Table of Contents](#Table-of-Contents)

## Appendix: R Session Info

In [87]:
Sys.time()
sessionInfo()

[1] "2023-03-03 10:48:37 MST"

R version 4.1.3 (2022-03-10)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur/Monterey 10.16

Matrix products: default
BLAS/LAPACK: /Users/dchilinfuentes/opt/anaconda3/envs/RNAseq_env/lib/libopenblasp-r0.3.20.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] edgeR_3.36.0                           
 [2] limma_3.50.3                           
 [3] Homo.sapiens_1.3.1                     
 [4] TxDb.Hsapiens.UCSC.hg19.knownGene_3.2.2
 [5] org.Hs.eg.db_3.14.0                    
 [6] GO.db_3.14.0                           
 [7] OrganismDbi_1.36.0                     
 [8] GenomicFeatures_1.46.5                 
 [9] GenomicRanges_1.46.1                   
[10] GenomeInfoDb_1.30.1                    
[11] AnnotationDbi_1.56.2                   
[12] IRanges_2.28.0                         
[13] S

[Table of Contents](#Table-of-Contents)

Copyright (c) 2018 UC San Diego Center for Computational Biology & Bioinformatics under the MIT License

Notebook template by Amanda Birmingham